## Question Answering with a fine-tuned BERT

### Importing the libraries

In [1]:
!pip install transformers #Huggingface
!pip install torch

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import torch #pytorch
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer

### Loading the pretrained models

In [3]:
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

### Function

In [4]:
'''
Input 1 = Question
Input 2 = Passage/Text
Output 1 = Answer

'''


def question_answer(question, text):
    
    #tokenize question and text in ids as a pair
    input_ids = tokenizer.encode(question, text)
    
    #string version of tokenized ids
    tokens = tokenizer.convert_ids_to_tokens(input_ids)
    
    #segment IDs
    #first occurence of [SEP] token
    sep_idx = input_ids.index(tokenizer.sep_token_id)

    #number of tokens in segment A - question
    num_seg_a = sep_idx+1

    #number of tokens in segment B - text
    num_seg_b = len(input_ids) - num_seg_a
    
    #list of 0s and 1s
    segment_ids = [0]*num_seg_a + [1]*num_seg_b
    
    assert len(segment_ids) == len(input_ids)
    
    #model output using input_ids and segment_ids
    output = model(torch.tensor([input_ids]), token_type_ids=torch.tensor([segment_ids]))
    
    #reconstructing the answer
    answer_start = torch.argmax(output.start_logits)
    answer_end = torch.argmax(output.end_logits)

    if answer_end >= answer_start:
        answer = tokens[answer_start]
        for i in range(answer_start+1, answer_end+1):
            if tokens[i][0:2] == "##":
                answer += tokens[i][2:]
            else:
                answer += " " + tokens[i]
                
    if answer.startswith("[CLS]"):
        answer = "Unable to find the answer to your question."
    
#     print("Text:\n{}".format(text.capitalize()))
#     print("\nQuestion:\n{}".format(question.capitalize()))
    print("\nAnswer:\n{}".format(answer.capitalize()))

In [11]:
text = """A neural network is a type of machine learning algorithm that is modeled after the structure of the human brain. 
It consists of layers of interconnected nodes or neurons, each of which performs a simple computation on its inputs. 
The output of each neuron is then passed to the next layer of neurons until a final output is produced. 
Neural networks are used for a wide range of tasks, including image and speech recognition, natural language processing, and predictive analytics. 
They are especially effective for tasks that involve complex patterns or non-linear relationships between inputs and outputs. 
One of the key advantages of neural networks is that they can learn and improve over time through a process called training. 
During training, the weights of the neurons are adjusted based on the error between the predicted output and the actual output, allowing the network to improve its accuracy and performance.
"""

question = "Can neural networks learn and improve over time?"

question_answer(question, text)


Answer:
Training


In [23]:
print("Original answer:\n", data.loc[data["question"] == question]["answer"].values[0])

Original answer:
 Hard Rock Cafe


### Playing with the chatbot

In [9]:
text = input("Please enter your text: \n")
question = input("\nPlease enter your question: \n")

while True:
    question_answer(question, text)
    
    flag = True
    flag_N = False
    
    while flag:
        response = input("\nDo you want to ask another question based on this text (Y/N)? ")
        if response[0] == "Y":
            question = input("\nPlease enter your question: \n")
            flag = False
        elif response[0] == "N":
            print("\nBye!")
            flag = False
            flag_N = True
            
    if flag_N == True:
        break


Answer:
4 – 2

Answer:
4 – 2


IndexError: string index out of range